In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import (
    mean_absolute_error, 
    mean_squared_error, 
    mean_absolute_percentage_error, 
    root_mean_squared_error
)

import xgboost as xgb
from xgboost import XGBRegressor, plot_importance
import lightgbm as lgb

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanAbsoluteError
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.python.client import device_lib

import keras_tuner as kt
from keras_tuner import Hyperband

from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll.base import scope

In [15]:
tf.__version__

'2.9.3'

In [16]:
def print_test_results(y_t, pred):
    print(f'[Test results]\n'
          f'MAPE: {round(mean_absolute_percentage_error(y_t, pred), 2)}\n'
          f'MAE: {round(mean_absolute_error(y_t, pred), 2)}\n'
          f'MSE: {round(mean_squared_error(y_t, pred), 2)}\n'
          f'RMSE: {round(root_mean_squared_error(y_t, pred), 2)}\n')

def print_test_results2(y_t, pred):
    print(f'[Test results]\n'
          f'MAPE: {round(mean_absolute_percentage_error(y_t, pred), 4)}\n'
          f'MAE: {round(mean_absolute_error(y_t, pred), 4)}\n'
          f'MSE: {"%.4e"%mean_squared_error(y_t, pred)}\n'
          f'MSE: {round(mean_squared_error(y_t, pred), 4)}\n'
          f'RMSE: {round(root_mean_squared_error(y_t, pred), 4)}\n')

In [17]:
raw_df = pd.read_excel('Data_평가만료일_특허유지일.xlsx')

In [18]:
df = raw_df.iloc[:,2:-1]
df

,출원인수,대리인수,패밀리특허수,IPC 수,청구항 수,독립청구항의 비율,출원-등록일까지의 기간(일),IPC_A,IPC_B,IPC_C,...,IPC활동성 평균(등록),IPC활동성 비율(등록),IPC 경쟁정도(등록),특허 권리이전 횟수,평가-만료일까지의 기간(일),피인용 수,피인용 특허 지수,TCT 지수,특허 청구항 지수,기술다양성 지수
0,1,1,8,1,6,0.666667,1266,0,0,0,...,239.8,0.712842,1770,4,162,0,0.000000,7,1.191367,0.386555
1,1,1,16,1,16,0.187500,930,0,0,0,...,834.8,0.815712,5438,2,162,15,2.146172,6,1.371265,0.614334
2,1,1,1,3,2,0.500000,1001,0,0,0,...,675.2,0.895016,3988,2,162,0,0.000000,6,0.235829,1.056587
3,1,3,9,1,5,1.000000,329,0,0,1,...,104.0,0.485528,1115,2,162,0,0.000000,8,0.455272,0.380000
4,1,3,7,1,9,0.333333,775,0,0,0,...,1871.2,0.708681,14210,0,162,0,0.000000,6,0.834862,0.733083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235772,1,1,13,1,18,0.111111,2629,0,0,0,...,2870.2,0.463279,33307,3,-1788,0,0.000000,6,1.057376,0.469958
235773,1,1,8,1,19,0.263158,2494,0,0,0,...,7022.4,0.542538,67836,3,233,3,0.602033,6,1.360796,0.495727
235774,1,1,18,1,1,1.000000,1288,0,0,1,...,47.4,0.429348,621,0,-497,0,0.000000,7,0.087488,0.534483
235775,1,1,25,1,23,0.173913,2228,0,0,1,...,176.8,0.460177,2060,2,-606,0,0.000000,9,1.503632,0.512658


In [19]:
X_train, X_test, y_train,y_test = train_test_split(df, raw_df.iloc[:,-1], test_size = 0.2, random_state=7)
train_cnt = y_train.count()
test_cnt = y_test.count()

In [20]:

# scaler = StandardScaler()
# scaler = MinMaxScaler()
scaler = RobustScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [21]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(188621, 27)
(47156, 27)
(188621,)
(47156,)


In [38]:
# GPU Test
print(tf.__version__)
print("==========================================")
print(tf.test.is_gpu_available())
print("==========================================")
print(tf.test.is_built_with_cuda())
print("==========================================")
print(tf.test.gpu_device_name())
print("==========================================")
device_lib.list_local_devices()

2.9.3
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True
True
/device:GPU:0


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14819614093422308885
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 10057940992
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13004242559693112844
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:01:00.0, compute capability: 8.6"
 xla_global_id: 416903419]

---

# 0. Build Model

## 0-1. RF

In [11]:
rf_model = RandomForestRegressor(n_estimators=280, 
                                 max_depth=30, 
                                 min_samples_split=2, 
                                 min_samples_leaf=1)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)

## 0-2. XGBoost

In [12]:
xgb_model = xgb.XGBRegressor(n_estimators=290, 
                             max_depth=14, 
                             learning_rate=0.03259162240984821, 
                             subsample=0.7206064233722951, 
                             colsample_bytree=0.9229737811346409)
xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)

## 0-3. DNN

In [13]:
units_1 = 128
dropout_1 = 0.3
units_2 = 96
dropout_2 = 0.2
units_3 = 56
dropout_3 = 0.5
units_4 = 24
dropout_4 = 0.2
learning_rate = 0.0012536297097257307
epochs = 200
initial_epoch = 67

# 모델 정의
def create_dnn_model():
    model = Sequential([
        Dense(units_1, activation='relu', input_shape=(X_train.shape[1],)),
        Dropout(dropout_1),
        Dense(units_2, activation='relu'),
        Dropout(dropout_2),
        Dense(units_3, activation='relu'),
        Dropout(dropout_3),
        Dense(units_4, activation='relu'),
        Dropout(dropout_4),
        Dense(1)
    ])
    return model

# 모델 생성
dnn_model = create_dnn_model()

# 옵티마이저 및 손실 함수 설정
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
dnn_model.compile(optimizer=optimizer, loss='mae', metrics=['mae'])

# 모델 학습
history = dnn_model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=epochs,
    initial_epoch=initial_epoch,
    batch_size=128
)

dnn_pred = dnn_model.predict(X_test)

Epoch 68/200
1474/1474 [==============================] - 3s 2ms/step - loss: 1186.1288 - mae: 1186.1288 - val_loss: 962.2440 - val_mae: 962.2440
Epoch 69/200
1474/1474 [==============================] - 2s 2ms/step - loss: 1053.8406 - mae: 1053.8406 - val_loss: 956.8305 - val_mae: 956.8305
Epoch 70/200
1474/1474 [==============================] - 2s 2ms/step - loss: 1046.4242 - mae: 1046.4242 - val_loss: 946.1011 - val_mae: 946.1011
Epoch 71/200
1474/1474 [==============================] - 2s 2ms/step - loss: 1041.5195 - mae: 1041.5195 - val_loss: 954.5012 - val_mae: 954.5012
Epoch 72/200
1474/1474 [==============================] - 2s 2ms/step - loss: 1038.3125 - mae: 1038.3125 - val_loss: 942.5836 - val_mae: 942.5836
Epoch 73/200
1474/1474 [==============================] - 2s 2ms/step - loss: 1038.7908 - mae: 1038.7908 - val_loss: 946.9658 - val_mae: 946.9658
Epoch 74/200
1474/1474 [==============================] - 2s 2ms/step - loss: 1037.0133 - mae: 1037.0133 - val_loss: 942.360

---

# 1. Stacking Ensemble

In [61]:
# y_test to df
df_y_test = y_test.reset_index(drop=True).to_frame()
df_y_test

# pred result to df
df_rf_predict = pd.DataFrame(rf_pred)
df_xgb_predict = pd.DataFrame(xgb_pred)
df_dnn_predict = pd.DataFrame(dnn_pred)

## 1-1. No K-Fold CV

In [62]:
df_concat_0 = pd.concat([df_rf_predict, df_dnn_predict, df_y_test], axis=1)
df_concat_0.columns = ['RF', 'DNN', 'True']

X0 = df_concat_0[['RF', 'DNN']]
y0 = df_concat_0['True']

print(X0.shape, type(X0))
print(y0.shape, type(y0))

(47156, 2) <class 'pandas.core.frame.DataFrame'>
(47156,) <class 'pandas.core.series.Series'>


In [63]:
X_train0, X_test0, y_train0, y_test0 = train_test_split(X0, y0, test_size=0.2, random_state=7)

scaler = StandardScaler()
X_train0 = scaler.fit_transform(X_train0)
X_test0 = scaler.transform(X_test0)

In [64]:
# DNN 모델 정의
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train0.shape[1],)),
    Dropout(0.3),  
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'), 
    Dense(1)
])

# 모델 컴파일
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss='mae',
                metrics=['mae'])

# 학습 (GPU 사용)
with tf.device("/device:GPU:0"):
    history = model.fit(X_train0, y_train0, 
                        epochs=200,  # 폴드마다 학습 횟수를 조정 가능
                        batch_size=128, 
                        validation_data=(X_test0, y_test0), 
                        verbose=1)

Epoch 1/200
295/295 [==============================] - 1s 2ms/step - loss: 1828.2286 - mae: 1828.2286 - val_loss: 955.7838 - val_mae: 955.7838
Epoch 2/200
295/295 [==============================] - 0s 2ms/step - loss: 911.1678 - mae: 911.1678 - val_loss: 867.5570 - val_mae: 867.5570
Epoch 3/200
295/295 [==============================] - 0s 2ms/step - loss: 888.8002 - mae: 888.8002 - val_loss: 861.1876 - val_mae: 861.1876
Epoch 4/200
295/295 [==============================] - 0s 2ms/step - loss: 886.2305 - mae: 886.2305 - val_loss: 859.9473 - val_mae: 859.9473
Epoch 5/200
295/295 [==============================] - 0s 1ms/step - loss: 886.2003 - mae: 886.2003 - val_loss: 862.2205 - val_mae: 862.2205
Epoch 6/200
295/295 [==============================] - 0s 1ms/step - loss: 884.3283 - mae: 884.3283 - val_loss: 858.7069 - val_mae: 858.7069
Epoch 7/200
295/295 [==============================] - 0s 1ms/step - loss: 886.7310 - mae: 886.7310 - val_loss: 858.3950 - val_mae: 858.3950
Epoch 8/200

In [65]:
no_kfold_pred = model.predict(X_test0)
print_test_results2(y_test0, no_kfold_pred)

295/295 [==============================] - 0s 671us/step
[Test results]
MAPE: 0.329
MAE: 864.6461
MSE: 1.2521e+06
MSE: 1252113.8208
RMSE: 1118.9789



---

## 1-2. RF + XGBoost + DNN

In [44]:
df_concat_1 = pd.concat([df_rf_predict, df_xgb_predict, df_dnn_predict, df_y_test], axis=1)
df_concat_1.columns = ['RF', 'DNN', 'XGBoost', 'True']

X1 = df_concat_1[['RF', 'DNN', 'XGBoost']]
y1 = df_concat_1['True']

print(X1.shape, type(X1))
print(y1.shape, type(y1))

(47156, 3) <class 'pandas.core.frame.DataFrame'>
(47156,) <class 'pandas.core.series.Series'>


In [47]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=7)

scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train1)
X_test1 = scaler.transform(X_test1)

In [48]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout


# K-Fold Cross Validation
def cross_validate_model(X_train, y_train, n_splits_list):
    for n_splits in n_splits_list:
        print(f"\n--- Cross Validation: {n_splits}-Fold 시작 ---\n")
        kfold = KFold(n_splits=n_splits, shuffle=True, random_state=7)

        fold_no = 1
        mae_per_fold = []
        
        for train_index, val_index in kfold.split(X_train):
            print(f"Fold {fold_no} / {n_splits}")
            
            # 데이터 분할
            X_fold_train, X_fold_val = X_train[train_index], X_train[val_index]
            y_fold_train, y_fold_val = y_train.iloc[train_index], y_train.iloc[val_index]
            
            # DNN 모델 정의
            model = Sequential([
                Dense(128, activation='relu', input_shape=(X_fold_train.shape[1],)),
                Dropout(0.3),  
                Dense(64, activation='relu'),
                Dropout(0.2),
                Dense(32, activation='relu'), 
                Dense(1)
            ])
            
            # 모델 컴파일
            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                          loss='mae',
                          metrics=['mae'])

            # 학습 (GPU 사용)
            with tf.device("/device:GPU:0"):
                history = model.fit(X_fold_train, y_fold_train, 
                                    epochs=100,  # 폴드마다 학습 횟수를 조정 가능
                                    batch_size=128, 
                                    validation_data=(X_fold_val, y_fold_val), 
                                    verbose=1)

                # 폴드별 평가
                loss, mae = model.evaluate(X_fold_val, y_fold_val, verbose=0)
                print(f"Fold {fold_no} - Loss: {loss:.4f}, MAE: {mae:.4f}")
                mae_per_fold.append(mae)

            fold_no += 1

        # 평균 MAE 출력
        print(f"\nCross Validation {n_splits}-Fold 결과:")
        print(f"MAE 평균: {np.mean(mae_per_fold):.4f}, 표준편차: {np.std(mae_per_fold):.4f}\n")

# Cross Validation 실행 (3, 5, 7-Fold)
n_splits_list = [3, 5, 7]
cross_validate_model(X_train1, y_train1, n_splits_list)


--- Cross Validation: 3-Fold 시작 ---

Fold 1 / 3
Epoch 1/100
197/197 [==============================] - 1s 2ms/step - loss: 2400.3469 - mae: 2400.3469 - val_loss: 1259.7412 - val_mae: 1259.7412
Epoch 2/100
197/197 [==============================] - 0s 1ms/step - loss: 1082.2333 - mae: 1082.2333 - val_loss: 951.2248 - val_mae: 951.2248
Epoch 3/100
197/197 [==============================] - 0s 1ms/step - loss: 916.3088 - mae: 916.3088 - val_loss: 870.2824 - val_mae: 870.2824
Epoch 4/100
197/197 [==============================] - 0s 1ms/step - loss: 891.4794 - mae: 891.4794 - val_loss: 862.1047 - val_mae: 862.1047
Epoch 5/100
197/197 [==============================] - 0s 2ms/step - loss: 887.8734 - mae: 887.8734 - val_loss: 859.6470 - val_mae: 859.6470
Epoch 6/100
197/197 [==============================] - 0s 2ms/step - loss: 884.9587 - mae: 884.9587 - val_loss: 859.9461 - val_mae: 859.9461
Epoch 7/100
197/197 [==============================] - 0s 2ms/step - loss: 887.0154 - mae: 887.0154

In [50]:
from sklearn.model_selection import KFold
import numpy as np

# 3-Fold Cross Validation
kfold = KFold(n_splits=3, shuffle=True, random_state=7)
mae_per_fold = []

fold_no = 1
for train_index, val_index in kfold.split(X_train1):
    print(f"Fold {fold_no} / 3")
    
    # 데이터 분할
    X_fold_train, X_fold_val = X_train1[train_index], X_train1[val_index]
    y_fold_train, y_fold_val = y_train1.iloc[train_index], y_train1.iloc[val_index]
    
    # 모델 정의
    model1 = Sequential([
        Dense(128, activation='relu', input_shape=(X_fold_train.shape[1],)),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(1)
    ])
    
    # 모델 컴파일
    model1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='mae',
                  metrics=['mae'])
    
    # 학습
    with tf.device("/device:GPU:0"):
        model1.fit(X_fold_train, y_fold_train,
                  epochs=200, 
                  batch_size=128, 
                  validation_data=(X_fold_val, y_fold_val), 
                  verbose=1)
        
        # 폴드별 평가
        loss, mae = model1.evaluate(X_fold_val, y_fold_val, verbose=0)
        print(f"Fold {fold_no} - Loss: {loss:.4f}, MAE: {mae:.4f}")
        mae_per_fold.append(mae)
    
    fold_pred = model1.predict(X_fold_val)
    print_test_results2(y_fold_val, fold_pred)
    
    fold_no += 1

# 평균 MAE와 표준편차 계산
print(f"\nCross Validation 3-Fold 결과:")
print(f"MAE 평균: {np.mean(mae_per_fold):.4f}, 표준편차: {np.std(mae_per_fold):.4f}\n")

# 최종 평가 (test set 사용)
ensemble_pred_1 = model1.predict(X_test1)
print_test_results2(y_test1, ensemble_pred_1)

Fold 1 / 3
Epoch 1/200
197/197 [==============================] - 1s 4ms/step - loss: 2232.7957 - mae: 2232.7957 - val_loss: 1201.2036 - val_mae: 1201.2036
Epoch 2/200
197/197 [==============================] - 0s 1ms/step - loss: 1044.5273 - mae: 1044.5273 - val_loss: 913.5385 - val_mae: 913.5385
Epoch 3/200
197/197 [==============================] - 0s 1ms/step - loss: 901.6115 - mae: 901.6115 - val_loss: 864.8330 - val_mae: 864.8330
Epoch 4/200
197/197 [==============================] - 0s 1ms/step - loss: 890.9505 - mae: 890.9505 - val_loss: 861.4249 - val_mae: 861.4249
Epoch 5/200
197/197 [==============================] - 0s 1ms/step - loss: 887.9727 - mae: 887.9727 - val_loss: 860.5688 - val_mae: 860.5688
Epoch 6/200
197/197 [==============================] - 0s 1ms/step - loss: 884.9762 - mae: 884.9762 - val_loss: 859.8094 - val_mae: 859.8094
Epoch 7/200
197/197 [==============================] - 0s 2ms/step - loss: 884.4091 - mae: 884.4091 - val_loss: 857.6708 - val_mae: 857.6

## 1-3. RF + DNN

In [66]:
df_concat_2 = pd.concat([df_rf_predict, df_dnn_predict, df_y_test], axis=1)
df_concat_2.columns = ['RF', 'DNN', 'True']

X2 = df_concat_2[['RF', 'DNN']]
y2 = df_concat_2['True']

print(X1.shape, type(X2))
print(y1.shape, type(y2))

(47156, 3) <class 'pandas.core.frame.DataFrame'>
(47156,) <class 'pandas.core.series.Series'>


In [67]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=7)

scaler = StandardScaler()
X_train2 = scaler.fit_transform(X_train2)
X_test2 = scaler.transform(X_test2)

### K-Fold Experiments(k=3, 5, 7)

In [69]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout



# K-Fold Cross Validation
def cross_validate_model(X_train, y_train, n_splits_list):
    for n_splits in n_splits_list:
        print(f"\n--- Cross Validation: {n_splits}-Fold 시작 ---\n")
        kfold = KFold(n_splits=n_splits, shuffle=True, random_state=7)

        fold_no = 1
        mae_per_fold = []
        mape_per_fold = []
        mse_per_fold = []
        rmse_per_fold = []
        
        for train_index, val_index in kfold.split(X_train):
            print(f"Fold {fold_no} / {n_splits}")
            
            # 데이터 분할
            X_fold_train, X_fold_val = X_train[train_index], X_train[val_index]
            y_fold_train, y_fold_val = y_train.iloc[train_index], y_train.iloc[val_index]
            
            # DNN 모델 정의
            model = Sequential([
                Dense(128, activation='relu', input_shape=(X_fold_train.shape[1],)),
                Dropout(0.3),  
                Dense(64, activation='relu'),
                Dropout(0.2),
                Dense(32, activation='relu'), 
                Dense(1)
            ])
            
            # 모델 컴파일
            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                          loss='mae',
                          metrics=['mae'])

            # 학습 (GPU 사용)
            with tf.device("/device:GPU:0"):
                history = model.fit(X_fold_train, y_fold_train, 
                                    epochs=100,  # 폴드마다 학습 횟수를 조정 가능
                                    batch_size=128, 
                                    validation_data=(X_fold_val, y_fold_val), 
                                    verbose=1)

                # 폴드별 평가
                loss, mae = model.evaluate(X_fold_val, y_fold_val, verbose=0)
                print(f"Fold {fold_no} - Loss: {loss:.4f}, MAE: {mae:.4f}")
                
                fold_pred = model.predict(X_fold_val)
                mae = round(mean_absolute_error(y_fold_val, fold_pred), 4)
                mape = round(mean_absolute_percentage_error(y_fold_val, fold_pred), 4)
                mse = round(mean_squared_error(y_fold_val, fold_pred), 4)
                rmse = round(root_mean_squared_error(y_fold_val, fold_pred), 4)
                
                mae_per_fold.append(mae)
                mape_per_fold.append(mape)
                mse_per_fold.append(mse)
                rmse_per_fold.append(rmse)                
        
            fold_no += 1
            
        print("[각 Fold별 평균 성능]")
        print(f"mae : {np.mean(mae_per_fold)}")
        print(f"mape : {np.mean(mape_per_fold)}")
        print(f"mse : {np.mean(mse_per_fold)}")
        print(f"rmse : {np.mean(rmse_per_fold)}")
        
        # 평균 MAE 출력
        # print(f"\nCross Validation {n_splits}-Fold 결과:")
        # print(f"MAE 평균: {np.mean(mae_per_fold):.4f}, 표준편차: {np.std(mae_per_fold):.4f}\n")

# Cross Validation 실행 (3, 5, 7-Fold)
n_splits_list = [3, 5, 7]
cross_validate_model(X_train2, y_train2, n_splits_list)


--- Cross Validation: 3-Fold 시작 ---

Fold 1 / 3
Epoch 1/100
197/197 [==============================] - 1s 2ms/step - loss: 2284.8145 - mae: 2284.8145 - val_loss: 1109.2766 - val_mae: 1109.2766
Epoch 2/100
197/197 [==============================] - 0s 2ms/step - loss: 993.6223 - mae: 993.6223 - val_loss: 895.7069 - val_mae: 895.7069
Epoch 3/100
197/197 [==============================] - 0s 1ms/step - loss: 899.2522 - mae: 899.2522 - val_loss: 867.3839 - val_mae: 867.3839
Epoch 4/100
197/197 [==============================] - 0s 1ms/step - loss: 885.8541 - mae: 885.8541 - val_loss: 858.9851 - val_mae: 858.9851
Epoch 5/100
197/197 [==============================] - 0s 1ms/step - loss: 891.9809 - mae: 891.9809 - val_loss: 857.6217 - val_mae: 857.6217
Epoch 6/100
197/197 [==============================] - 0s 1ms/step - loss: 887.2106 - mae: 887.2106 - val_loss: 856.1957 - val_mae: 856.1957
Epoch 7/100
197/197 [==============================] - 0s 2ms/step - loss: 884.3329 - mae: 884.3329 -

### Fixed K-Fold (K=5)

In [70]:
from sklearn.model_selection import KFold
import numpy as np

# 3-Fold Cross Validation
kfold = KFold(n_splits=5, shuffle=True, random_state=7)
mae_per_fold = []
mape_per_fold = []
mse_per_fold = []
rmse_per_fold = []

fold_no = 1
for train_index, val_index in kfold.split(X_train2):
    print(f"Fold {fold_no} / 5")
    
    # 데이터 분할
    X_fold_train, X_fold_val = X_train2[train_index], X_train2[val_index]
    y_fold_train, y_fold_val = y_train2.iloc[train_index], y_train2.iloc[val_index]
    
    # 모델 정의
    model2 = Sequential([
        Dense(128, activation='relu', input_shape=(X_fold_train.shape[1],)),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(1)
    ])
    
    # 모델 컴파일
    model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='mae',
                  metrics=['mae'])
    
    # 학습
    with tf.device("/device:GPU:0"):
        model2.fit(X_fold_train, y_fold_train,
                  epochs=200, 
                  batch_size=128, 
                  validation_data=(X_fold_val, y_fold_val), 
                  verbose=1)
        
        # 폴드별 평가
        loss, mae = model2.evaluate(X_fold_val, y_fold_val, verbose=0)
        print(f"Fold {fold_no} - Loss: {loss:.4f}, MAE: {mae:.4f}")
        mae_per_fold.append(mae)
    
    fold_pred = model2.predict(X_fold_val)
    mae = round(mean_absolute_error(y_fold_val, fold_pred), 4)
    mape = round(mean_absolute_percentage_error(y_fold_val, fold_pred), 4)
    mse = round(mean_squared_error(y_fold_val, fold_pred), 4)
    rmse = round(root_mean_squared_error(y_fold_val, fold_pred), 4)
    mae_per_fold.append(mae)
    mape_per_fold.append(mape)
    mse_per_fold.append(mse)
    rmse_per_fold.append(rmse)
    
    fold_no += 1

# 평균 MAE와 표준편차 계산
print(f"\nCross Validation 5-Fold 결과:")
print(f"MAE 평균: {np.mean(mae_per_fold):.4f}, 표준편차: {np.std(mae_per_fold):.4f}\n")

print("[5-Fold 평균 성능]")
print(f"mae : {np.mean(mae_per_fold)}")
print(f"mape : {np.mean(mape_per_fold)}")
print(f"mse : {np.mean(mse_per_fold)}")
print(f"rmse : {np.mean(rmse_per_fold)}")
# 최종 평가 (test set 사용)
ensemble_pred_2 = model2.predict(X_test2)
print_test_results2(y_test2, ensemble_pred_2)

Fold 1 / 5
Epoch 1/200
236/236 [==============================] - 1s 2ms/step - loss: 2064.5725 - mae: 2064.5725 - val_loss: 1042.5881 - val_mae: 1042.5881
Epoch 2/200
236/236 [==============================] - 0s 1ms/step - loss: 952.5894 - mae: 952.5894 - val_loss: 878.0933 - val_mae: 878.0933
Epoch 3/200
236/236 [==============================] - 0s 1ms/step - loss: 890.9266 - mae: 890.9266 - val_loss: 861.1250 - val_mae: 861.1250
Epoch 4/200
236/236 [==============================] - 0s 1ms/step - loss: 886.7127 - mae: 886.7127 - val_loss: 858.8304 - val_mae: 858.8304
Epoch 5/200
236/236 [==============================] - 0s 1ms/step - loss: 887.3256 - mae: 887.3256 - val_loss: 861.3793 - val_mae: 861.3793
Epoch 6/200
236/236 [==============================] - 0s 1ms/step - loss: 884.9254 - mae: 884.9254 - val_loss: 863.4688 - val_mae: 863.4688
Epoch 7/200
236/236 [==============================] - 0s 1ms/step - loss: 888.4480 - mae: 888.4480 - val_loss: 858.0247 - val_mae: 858.024